In [1]:
#imports
import pandas as pd
from generate_card_text import CardTextGenerator
import ipywidgets as widgets

In [2]:
# pandas options for bigger grids
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [3]:
# Download and load the model. 
ai = CardTextGenerator()

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading file https://huggingface.co/minimaxir/magic-the-gathering/resolve/main/vocab.json from cache at None
loading file https://huggingface.co/minimaxir/magic-the-gathering/resolve/main/merges.txt from cache at None
loading file https://huggingface.co/minimaxir/magic-the-gathering/resolve/main/tokenizer.json from cache at aitextgen/b1431d1de2fe5eabc1480965cc002593e0a1e07de5baaabbb66474fa2b5770d8.862a5b76401704bc81e91e440c522f95164df652d01d290de56ebc2896c47a98
loading file https://huggingface.co/minimaxir/magic-the-gathering/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/minimaxir/magic-the-gathering/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/minimaxir/magic-the-gathering/resolve/main/tokenizer_config.json from cache at None


In [4]:
class CardsStorage:
    def __init__(self):
        self.current_cards = None
        self.selected_cards = pd.DataFrame(columns=
        ['name', 'manaCost', 'type', 'power', 'toughness','loyalty','text'])

store = CardsStorage()

In [5]:
def generate_card_details(_):
    prompt=promptInput.value
    craziness = crazinessSlider.value
    numOfCards = batch_size = 3
    store.current_cards = ai.generate_card(
    prompt,
    craziness, 
    numOfCards,
    batch_size)
    cardTable.value = store.current_cards.to_html()

In [6]:
from time import sleep
def add_selected_card(_):
    selected_card = store.current_cards.iloc[selectedCardInput.value - 1]
    selected_card_as_df = pd.DataFrame.from_records([selected_card.to_dict()])
    store.selected_cards = pd.concat([store.selected_cards, selected_card_as_df])
    cardAddedLabel.value = "   card added!"
    sleep(1)
    cardAddedLabel.value = ""

In [7]:
from ipywidgets import HBox, VBox, Label, Layout
wider = Layout(width="70vw")
less_wide = Layout(width="5vw")

In [8]:
promptInput = widgets.Text(placeholder='ex: <|type|>Creature - Goblin<|manaCost|>{2}{R}', layout=wider)
crazinessSlider = widgets.FloatSlider(.7,min=0.5,max=1.3)

generateCardsButton = widgets.Button(description="Generate")
generateCardsButton.on_click(generate_card_details)

cardTable = widgets.HTML()

In [9]:
selectedCardInput = widgets.BoundedIntText(1,min=1,max=3,layout=less_wide)
addCardButton = widgets.Button(description="add to deck")
addCardButton.on_click(add_selected_card)
cardAddedLabel = Label()

In [10]:
display(HBox([Label("custom card prompt:"), promptInput]))
display(VBox([
    Label("(prompt list: <|name|>, <|manaCost|>, <|type|>, <|text|>,"),
    Label("<|power|>, <|toughness|>, <|loyalty|>)")
]))
display(HBox([Label("craziness:"), crazinessSlider]))
display(generateCardsButton)

display(cardTable)

display(HBox([selectedCardInput,addCardButton,cardAddedLabel]))

Button(description='Generate', style=ButtonStyle())

HTML(value='')

Run the cell bellow again to see the cards you selected

In [11]:
print("your cards:")
display(store.selected_cards)

your cards:


,name,manaCost,type,power,toughness,loyalty,text


In [12]:
saveCardsBtn = widgets.Button(description="save cards")
fileLocationInput = widgets.Text(value="My Cards.csv")

def saveCards(_):
    store.selected_cards.to_csv(f'../output/{fileLocationInput.value}')

saveCardsBtn.on_click(saveCards)
display(HBox([fileLocationInput, saveCardsBtn]))